# pH-Sensitive Binder Design Evaluation

## Overview
This task evaluates your understanding and implementation of computational workflows for designing pH-sensitive protein binders, based on the strategies described in Ahn et al. 2025 "Computational Design of pH-Sensitive Binders".

## Two Design Strategies

### Strategy A: Interface Destabilization
- **Goal**: Weaken binder-target interaction at low pH
- **Mechanism**: Place histidine residues adjacent to positively charged residues (Arg/Lys) at the binding interface
- **Effect**: At low pH (~5.5), His becomes protonated (+1 charge), causing electrostatic repulsion with nearby Arg/Lys

### Strategy B: Monomer Destabilization  
- **Goal**: Destabilize the binder protein fold at low pH
- **Mechanism**: Install buried His-containing hydrogen bond networks (His-His, His-Arg, His-Lys) in the protein core
- **Effect**: At low pH, His protonation disrupts these buried H-bond networks, destabilizing the protein

## Task Structure
- **Part 0**: Conceptual understanding (multiple choice)
- **Part 1**: ProteinMPNN with histidine bias
- **Part 2**: Interface destabilization scoring & filtering (Strategy A)
- **Part 3**: Monomer destabilization with HBNet (Strategy B)


# Part 0: Conceptual Understanding

Answer the following multiple choice questions about pH-sensitive binder design.


## Question 0.1
**Why is histidine (His) the key residue for designing pH-sensitive proteins?**

A) His has the largest side chain among amino acids  
B) His has a pKa (~6.0) near physiological/endosomal pH, allowing protonation state changes  
C) His forms the strongest hydrogen bonds  
D) His is the most hydrophobic amino acid  

**Your answer:**


## Question 0.2
**In Strategy A (Interface Destabilization), which residue pairs create the pH-sensitive motif?**

A) His adjacent to Asp/Glu (negatively charged residues)  
B) His adjacent to Arg/Lys (positively charged residues)  
C) His adjacent to Ser/Thr (polar residues)  
D) His adjacent to Ala/Val (hydrophobic residues)  

**Your answer:**


## Question 0.3
**In Strategy B (Monomer Destabilization), where should the His-containing hydrogen bond networks be placed?**

A) At the protein surface, exposed to solvent  
B) At the binding interface with the target  
C) Buried in the protein core  
D) At the N-terminus of the protein  

**Your answer:**


## Question 0.4
**What network types are valid for Strategy B (Monomer Destabilization)?**

A) His-His, His-Arg, His-Lys networks  
B) Cys-Cys disulfide bonds  
C) Salt bridges between Asp and Lys  
D) Hydrophobic packing of Leu/Ile/Val  

**Your answer:**


---
# Part 1: ProteinMPNN with Histidine Bias

## Objective
Design sequences for a binder backbone using ProteinMPNN with histidine bias at specified interface positions.

## Background
ProteinMPNN is a deep learning method for protein sequence design. It can accept amino acid biases to favor certain residues at specific positions. For pH-sensitive binder design, we want to bias histidine at interface positions to enable Strategy A (Interface Destabilization).

## Task
1. Load the example binder backbone and target protein (IL-6)
2. Identify interface residues (within 8Å of target)
3. Run ProteinMPNN with histidine bias at interface positions
4. Analyze the designed sequences for His enrichment


In [ ]:
# Setup and imports
import os
import json
import numpy as np
from collections import Counter

# Check if ProteinMPNN is available
try:
    # ProteinMPNN can be called via subprocess or API
    PROTEINMPNN_AVAILABLE = True
    print("ProteinMPNN: Ready to use")
except:
    PROTEINMPNN_AVAILABLE = False
    print("ProteinMPNN: Not available - install from https://github.com/dauparas/ProteinMPNN")

# Data paths
DATA_DIR = "data"
TARGET_IL6 = os.path.join(DATA_DIR, "target_il6.pdb")  # PDB 1ALU

print(f"Target structure: {TARGET_IL6}")
print(f"File exists: {os.path.exists(TARGET_IL6)}")


## Task 1.1: Understanding ProteinMPNN Histidine Bias

ProteinMPNN supports amino acid biases through the `--bias_AA` flag or position-specific biases via `--bias_AA_per_residue`.

**Question**: Write the command-line argument to bias histidine with a log-odds score of +2.0 globally.

```bash
# Fill in the blank:
python ProteinMPNN/protein_mpnn_run.py \
    --pdb_path binder_complex.pdb \
    --out_folder output/ \
    ____________________________  # Add His bias argument here
```


In [ ]:
# Task 1.1: Your answer - write the bias argument
# Hint: The format is --bias_AA "X:value" where X is the amino acid letter

BIAS_ARGUMENT = ""  # TODO: Fill in the correct argument

print(f"Your bias argument: {BIAS_ARGUMENT}")


## Task 1.2: Position-Specific Histidine Bias

For more precise control, you can specify per-residue biases using a JSON file.

**Question**: Create a JSON dictionary that biases positions 10, 15, and 22 toward histidine (H) with a bias of +3.0, while keeping all other amino acids at 0.0 bias.


In [ ]:
# Task 1.2: Create position-specific bias dictionary
# The format should be: {chain_residue: {amino_acid: bias_value, ...}, ...}
# Example: {"A10": {"H": 3.0, "A": 0.0, ...}, "A15": {...}, "A22": {...}}

# Amino acid alphabet
AA_ALPHABET = "ACDEFGHIKLMNPQRSTVWY"

def create_his_bias_dict(positions, chain="A", his_bias=3.0):
    """
    Create a per-residue bias dictionary for ProteinMPNN.
    
    Args:
        positions: List of residue numbers to bias toward His
        chain: Chain identifier
        his_bias: Log-odds bias for histidine
    
    Returns:
        Dictionary in ProteinMPNN bias format:
        {
            "A10": {"A": 0.0, "C": 0.0, ..., "H": 3.0, ..., "Y": 0.0},
            ...
        }
    """
    bias_dict = {}
    
    # TODO: Implement this function
    # For each position, create a dictionary with His biased and others at 0
    
    return bias_dict

# Test with positions 10, 15, 22
interface_positions = [10, 15, 22]
bias_dict = create_his_bias_dict(interface_positions, his_bias=3.0)

print("Position-specific bias dictionary:")
print(json.dumps(bias_dict, indent=2))

# Verify structure
if bias_dict:
    print(f"\nNumber of positions biased: {len(bias_dict)}")
    for key in bias_dict:
        print(f"  {key}: His bias = {bias_dict[key].get('H', 'N/A')}, num AAs = {len(bias_dict[key])}")


## Task 1.3: Interface Residue Selection for His Bias

Before applying histidine bias, we need to identify which residues are at the binder-target interface. Interface residues are defined as binder residues with at least one heavy atom within a distance cutoff (typically 8Å) of any target atom.

**Task**: Implement a function to identify interface residues from a binder-target complex structure.


In [ ]:
# Task 1.3: Interface residue selection
from Bio.PDB import PDBParser, NeighborSearch, Selection

def get_interface_residues(pdb_path, binder_chain="A", target_chain="B", distance_cutoff=8.0):
    """
    Identify interface residues on the binder chain based on distance to target.
    
    Interface residues are defined as binder residues with at least one heavy atom
    within distance_cutoff of any target atom.
    
    Args:
        pdb_path: Path to PDB file containing binder-target complex
        binder_chain: Chain ID of the binder protein
        target_chain: Chain ID of the target protein  
        distance_cutoff: Distance threshold in Angstroms (default 8.0Å)
    
    Returns:
        List of residue numbers that are at the interface
    """
    interface_residues = []
    
    # TODO: Implement this function
    # Steps:
    # 1. Parse the PDB structure
    # 2. Get all atoms from the target chain
    # 3. Build a NeighborSearch tree from target atoms
    # 4. For each residue in binder chain, check if any atom is within cutoff of target
    # 5. Return list of interface residue numbers
    
    return interface_residues


# =============================================================================
# REFERENCE IMPLEMENTATION (for testing/validation)
# =============================================================================
def get_interface_residues_reference(pdb_path, binder_chain="A", target_chain="B", distance_cutoff=8.0):
    """Reference implementation for interface residue detection."""
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("complex", pdb_path)
    model = structure[0]
    
    # Check if chains exist
    chain_ids = [c.id for c in model.get_chains()]
    if binder_chain not in chain_ids:
        raise ValueError(f"Binder chain '{binder_chain}' not found. Available: {chain_ids}")
    if target_chain not in chain_ids:
        raise ValueError(f"Target chain '{target_chain}' not found. Available: {chain_ids}")
    
    # Get target atoms for neighbor search (heavy atoms only)
    target_atoms = [atom for atom in model[target_chain].get_atoms() 
                    if atom.element != 'H']
    
    if len(target_atoms) == 0:
        raise ValueError(f"No atoms found in target chain '{target_chain}'")
    
    ns = NeighborSearch(target_atoms)
    
    interface_residues = []
    
    # Check each binder residue
    for residue in model[binder_chain].get_residues():
        # Skip hetero residues (water, ligands)
        if residue.id[0] != ' ':
            continue
            
        # Check if any heavy atom in this residue is near target
        is_interface = False
        for atom in residue.get_atoms():
            if atom.element == 'H':
                continue
            nearby = ns.search(atom.coord, distance_cutoff)
            if len(nearby) > 0:
                is_interface = True
                break
        
        if is_interface:
            interface_residues.append(residue.id[1])
    
    return sorted(interface_residues)


# Test with target_il6.pdb (has chain A)
print("Testing interface residue detection...")
print(f"Target structure: {TARGET_IL6}")

if os.path.exists(TARGET_IL6):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("il6", TARGET_IL6)
    chains = [c.id for c in structure[0].get_chains()]
    print(f"Available chains: {chains}")
    
    # For single-chain structures, we can demonstrate with self-interface (for testing)
    if len(chains) >= 2:
        try:
            interface = get_interface_residues_reference(TARGET_IL6, chains[0], chains[1], 8.0)
            print(f"\nInterface residues (chain {chains[0]} near chain {chains[1]}):")
            print(f"  Count: {len(interface)}")
            print(f"  Positions: {interface[:20]}{'...' if len(interface) > 20 else ''}")
        except Exception as e:
            print(f"Error: {e}")
    else:
        print(f"\nNote: Single chain structure. For testing interface detection,")
        print(f"      you need a binder-target complex with two chains.")
else:
    print(f"File not found: {TARGET_IL6}")


## Task 1.4: Complete Interface His-Bias Workflow

Combine interface selection with His bias creation for ProteinMPNN redesign.


In [ ]:
# Task 1.4: Complete workflow - Interface selection + His bias for ProteinMPNN

def create_interface_his_bias(pdb_path, binder_chain="A", target_chain="B", 
                               distance_cutoff=8.0, his_bias=3.0):
    """
    Complete workflow: Identify interface residues and create His bias dictionary.
    
    Args:
        pdb_path: Path to binder-target complex PDB
        binder_chain: Chain ID of binder
        target_chain: Chain ID of target
        distance_cutoff: Distance threshold for interface definition (Å)
        his_bias: Log-odds bias value for histidine
    
    Returns:
        Tuple of (interface_positions, bias_dict)
    """
    # Step 1: Get interface residues
    interface_positions = get_interface_residues(
        pdb_path, binder_chain, target_chain, distance_cutoff
    )
    
    print(f"Found {len(interface_positions)} interface residues")
    print(f"Interface positions: {interface_positions}")
    
    # Step 2: Create His bias dictionary for these positions
    bias_dict = create_his_bias_dict(interface_positions, chain=binder_chain, his_bias=his_bias)
    
    # Step 3: Validate reasonable interface size
    if len(interface_positions) < 5:
        print("WARNING: Very small interface detected. Check chain IDs and structure.")
    elif len(interface_positions) > 50:
        print("WARNING: Very large interface detected. Consider using smaller cutoff.")
    else:
        print(f"Interface size looks reasonable ({len(interface_positions)} residues)")
    
    return interface_positions, bias_dict


def save_bias_json(bias_dict, output_path):
    """Save bias dictionary to JSON file for ProteinMPNN."""
    with open(output_path, 'w') as f:
        json.dump(bias_dict, f, indent=2)
    print(f"Saved bias dictionary to: {output_path}")


# =============================================================================
# REFERENCE IMPLEMENTATION for complete workflow
# =============================================================================
def create_interface_his_bias_reference(pdb_path, binder_chain="A", target_chain="B", 
                                         distance_cutoff=8.0, his_bias=3.0):
    """Reference implementation of complete workflow."""
    # Step 1: Get interface residues using reference implementation
    interface_positions = get_interface_residues_reference(
        pdb_path, binder_chain, target_chain, distance_cutoff
    )
    
    print(f"Found {len(interface_positions)} interface residues")
    print(f"Interface positions: {interface_positions[:15]}{'...' if len(interface_positions) > 15 else ''}")
    
    # Step 2: Create His bias dictionary
    bias_dict = {}
    for pos in interface_positions:
        key = f"{binder_chain}{pos}"
        bias_dict[key] = {aa: (his_bias if aa == "H" else 0.0) for aa in AA_ALPHABET}
    
    # Step 3: Validate
    if len(interface_positions) < 5:
        print("⚠️  WARNING: Very small interface detected.")
    elif len(interface_positions) > 50:
        print("⚠️  WARNING: Very large interface. Consider smaller cutoff.")
    else:
        print(f"✓ Interface size looks reasonable ({len(interface_positions)} residues)")
    
    return interface_positions, bias_dict


# Example usage
print("Complete interface His-bias workflow defined.")
print("\nExpected workflow:")
print("  1. Load binder-target complex structure")
print("  2. Identify interface residues (within 8Å of target)")
print("  3. Create position-specific His bias for interface residues")
print("  4. Save bias JSON for ProteinMPNN")
print("\nTypical interface: 15-30 residues for reasonable binder size")


---
# Part 2: Interface Destabilization Scoring & Filtering (Strategy A)

## Objective
Implement a comprehensive scoring and filtering pipeline to identify promising Interface Destabilization candidates.

## Filtering Criteria (from paper workflow)
1. **His-cation motif**: At least one His accepting H-bond from Arg/Lys at interface
2. **Rosetta ddG_elec >= 0**: Electrostatic energy should be unfavorable at low pH (protonated His)
3. **Additional metrics**: dSASA_int, packstat, total_score after FastRelax

## Data
- Target: PCSK9 (PDB 2P4E)


In [ ]:
# Part 2: Setup PyRosetta
import warnings
warnings.filterwarnings('ignore')

# Initialize PyRosetta (uncomment when running)
# import pyrosetta
# pyrosetta.init("-mute all")

# For demonstration, we'll use BioPython for structure analysis
from Bio.PDB import PDBParser, NeighborSearch, Selection
import os

TARGET_PCSK9 = os.path.join(DATA_DIR, "target_pcsk9.pdb")
print(f"Target PCSK9: {TARGET_PCSK9}")
print(f"File exists: {os.path.exists(TARGET_PCSK9)}")


## Task 2.1: Detect His-Cation Interface Motif

Implement a function to detect histidine residues that are:
1. At the binding interface (within 5Å of target)
2. Within H-bond distance (< 3.5Å) of a positively charged residue (Arg or Lys)


In [ ]:
def detect_his_cation_motif(structure, binder_chain="A", target_chain="B", 
                            interface_cutoff=5.0, hbond_cutoff=3.5):
    """
    Detect His-Arg/Lys motifs at the binding interface.
    
    Args:
        structure: BioPython Structure object
        binder_chain: Chain ID of the binder
        target_chain: Chain ID of the target
        interface_cutoff: Distance cutoff for interface residues (Å)
        hbond_cutoff: Distance cutoff for H-bond detection (Å)
    
    Returns:
        List of dictionaries containing His-cation pairs found:
        [
            {
                'his_chain': str, 'his_resnum': int, 'his_atom': str,
                'cation_chain': str, 'cation_resnum': int, 'cation_resname': str,
                'cation_atom': str, 'distance': float
            },
            ...
        ]
    """
    his_cation_pairs = []
    
    # TODO: Implement this function
    # Steps:
    # 1. Get all atoms from binder and target chains
    # 2. Find interface residues (binder residues within interface_cutoff of target)
    # 3. For each His at interface, find nearby Arg/Lys within hbond_cutoff
    # 4. Check if the His nitrogen atoms (ND1, NE2) are within H-bond distance
    
    return his_cation_pairs


# =============================================================================
# REFERENCE IMPLEMENTATION
# =============================================================================
# Cation atoms in Arg and Lys that can donate H-bonds to His
CATION_DONORS = {
    'ARG': ['NH1', 'NH2', 'NE'],  # Guanidinium nitrogens
    'LYS': ['NZ'],                 # Epsilon amino nitrogen
}

# His acceptor atoms
HIS_ACCEPTORS = ['ND1', 'NE2']  # Imidazole nitrogens


def detect_his_cation_motif_reference(pdb_path, binder_chain="A", target_chain="B",
                                       interface_cutoff=5.0, hbond_cutoff=3.5):
    """
    Reference implementation for His-cation motif detection.
    
    Detects His residues at the interface that could form H-bonds with 
    positively charged Arg/Lys residues.
    """
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("complex", pdb_path)
    model = structure[0]
    
    # Get all chains in structure
    chain_ids = [c.id for c in model.get_chains()]
    
    his_cation_pairs = []
    
    # Collect all His residues and their acceptor atoms
    his_atoms = []
    for chain in model.get_chains():
        for residue in chain.get_residues():
            if residue.resname == 'HIS' and residue.id[0] == ' ':
                for atom_name in HIS_ACCEPTORS:
                    if atom_name in residue:
                        his_atoms.append({
                            'atom': residue[atom_name],
                            'chain': chain.id,
                            'resnum': residue.id[1],
                            'atom_name': atom_name
                        })
    
    # Collect all cation (Arg/Lys) donor atoms
    cation_atoms = []
    for chain in model.get_chains():
        for residue in chain.get_residues():
            if residue.resname in CATION_DONORS and residue.id[0] == ' ':
                for atom_name in CATION_DONORS[residue.resname]:
                    if atom_name in residue:
                        cation_atoms.append({
                            'atom': residue[atom_name],
                            'chain': chain.id,
                            'resnum': residue.id[1],
                            'resname': residue.resname,
                            'atom_name': atom_name
                        })
    
    # Find His-cation pairs within H-bond distance
    for his in his_atoms:
        for cation in cation_atoms:
            # Skip same residue
            if his['chain'] == cation['chain'] and his['resnum'] == cation['resnum']:
                continue
            
            # Calculate distance
            dist = his['atom'] - cation['atom']
            
            if dist <= hbond_cutoff:
                his_cation_pairs.append({
                    'his_chain': his['chain'],
                    'his_resnum': his['resnum'],
                    'his_atom': his['atom_name'],
                    'cation_chain': cation['chain'],
                    'cation_resnum': cation['resnum'],
                    'cation_resname': cation['resname'],
                    'cation_atom': cation['atom_name'],
                    'distance': round(dist, 2)
                })
    
    return his_cation_pairs


# Test with available structures
print("Testing His-cation motif detection...")

if os.path.exists(TARGET_PCSK9):
    pairs = detect_his_cation_motif_reference(TARGET_PCSK9, hbond_cutoff=3.5)
    print(f"\nPCSK9 structure ({TARGET_PCSK9}):")
    print(f"  Found {len(pairs)} His-cation pairs within 3.5Å")
    
    if pairs:
        print("\n  Top pairs:")
        for pair in pairs[:5]:
            print(f"    HIS {pair['his_chain']}{pair['his_resnum']}:{pair['his_atom']} - "
                  f"{pair['cation_resname']} {pair['cation_chain']}{pair['cation_resnum']}:{pair['cation_atom']} "
                  f"({pair['distance']:.2f}Å)")
else:
    print(f"File not found: {TARGET_PCSK9}")


## Task 2.2: PyRosetta FastRelax and Interface Analysis

Implement the Rosetta-based scoring pipeline:
1. FastRelax the binder-target complex
2. Calculate interface energy metrics using InterfaceAnalyzer
3. Extract fa_elec (electrostatic) energy decomposition


In [ ]:
def score_interface_destabilization(pdb_path, binder_chain="A", target_chain="B"):
    """
    Score a binder-target complex for interface destabilization potential.
    
    Uses PyRosetta to:
    1. FastRelax the complex
    2. Calculate interface metrics using InterfaceAnalyzerMover
    3. Decompose electrostatic energies for ddG_elec calculation
    
    Args:
        pdb_path: Path to binder-target complex PDB
        binder_chain: Chain ID of binder (e.g., "A")
        target_chain: Chain ID of target (e.g., "B")
    
    Returns:
        Dictionary of scores:
        {
            'total_score': float,      # Total Rosetta energy after relaxation
            'dG_separated': float,     # Binding energy (kcal/mol)
            'dSASA_int': float,        # Buried surface area at interface (Å²)
            'fa_elec': float,          # Electrostatic energy component
            'packstat': float,         # Packing quality (0-1)
            'ddG_elec': float          # Electrostatic ddG (should be >=0 for candidates)
        }
    
    Note:
        Requires PyRosetta to be installed and initialized.
        Run: pyrosetta.init("-mute all") before calling.
    
    Example:
        >>> import pyrosetta
        >>> pyrosetta.init("-mute all")
        >>> scores = score_interface_destabilization("complex.pdb", "A", "B")
        >>> if scores['ddG_elec'] >= 0:
        ...     print("Candidate passes electrostatic criterion")
    """
    scores = {
        'total_score': None,
        'dG_separated': None,
        'dSASA_int': None,
        'fa_elec': None,
        'packstat': None,
        'ddG_elec': None
    }
    
    # TODO: Implement using PyRosetta
    # Required steps:
    # 1. Load pose from PDB
    # 2. Setup score function (ref2015 or similar)
    # 3. Apply FastRelax protocol
    # 4. Run InterfaceAnalyzerMover with interface definition (e.g., "A_B")
    # 5. Extract scores from pose.scores dictionary
    # 6. Calculate ddG_elec (electrostatic contribution to binding)
    
    raise NotImplementedError(
        "This function requires PyRosetta. "
        "Install from: https://www.pyrosetta.org/downloads"
    )
    
    return scores


# =============================================================================
# STUB for PyRosetta scoring (returns mock data for testing pipeline)
# =============================================================================
def score_interface_destabilization_stub(pdb_path, binder_chain="A", target_chain="B"):
    """
    Stub implementation returning mock scores for pipeline testing.
    Replace with actual PyRosetta implementation.
    """
    import random
    return {
        'total_score': random.uniform(-200, -100),
        'dG_separated': random.uniform(-20, -5),
        'dSASA_int': random.uniform(800, 1500),
        'fa_elec': random.uniform(-10, 5),
        'packstat': random.uniform(0.5, 0.8),
        'ddG_elec': random.uniform(-2, 3)  # Positive = destabilized at low pH
    }

print("PyRosetta scoring function defined (stub available for testing).")
print("\nKey metrics for interface destabilization:")
print("  - ddG_elec >= 0: Electrostatic destabilization at low pH")
print("  - dSASA_int: Buried surface area (larger = better interface)")
print("  - packstat: Packing quality (0.6-0.8 typical for good designs)")


In [ ]:
def filter_interface_destabilization_candidates(pdb_files, 
                                                 min_his_cation_pairs=1,
                                                 min_ddg_elec=0.0):
    """
    Filter binder candidates for interface destabilization strategy.
    
    Criteria:
    1. At least min_his_cation_pairs His-Arg/Lys pairs at interface
    2. ddG_elec >= min_ddg_elec (electrostatic penalty at low pH)
    
    Args:
        pdb_files: List of PDB file paths to evaluate
        min_his_cation_pairs: Minimum number of His-cation pairs required
        min_ddg_elec: Minimum electrostatic ddG threshold
    
    Returns:
        List of passing candidates with their analysis:
        [
            {
                'pdb_path': str,
                'his_cation_pairs': list,
                'num_pairs': int,
                'scores': dict,
                'passes_motif': bool,
                'passes_elec': bool,
                'passes_all': bool
            },
            ...
        ]
    """
    passing_candidates = []
    
    for pdb_path in pdb_files:
        # TODO: Implement filtering logic
        # 1. Load structure and detect His-cation motifs
        # 2. Score with PyRosetta (or stub)
        # 3. Check both criteria
        # 4. Record results
        pass
    
    return passing_candidates


# =============================================================================
# REFERENCE IMPLEMENTATION for filtering pipeline
# =============================================================================
def filter_interface_destabilization_candidates_reference(
    pdb_files, 
    min_his_cation_pairs=1,
    min_ddg_elec=0.0,
    hbond_cutoff=3.5,
    use_stub=True  # Use stub for PyRosetta scoring
):
    """
    Reference implementation of the filtering pipeline.
    
    Combines His-cation motif detection with scoring to filter candidates.
    """
    results = []
    
    for pdb_path in pdb_files:
        if not os.path.exists(pdb_path):
            print(f"  Skipping (not found): {pdb_path}")
            continue
        
        result = {
            'pdb_path': pdb_path,
            'his_cation_pairs': [],
            'num_pairs': 0,
            'scores': {},
            'passes_motif': False,
            'passes_elec': False,
            'passes_all': False
        }
        
        # Step 1: Detect His-cation motifs
        try:
            pairs = detect_his_cation_motif_reference(pdb_path, hbond_cutoff=hbond_cutoff)
            result['his_cation_pairs'] = pairs
            result['num_pairs'] = len(pairs)
            result['passes_motif'] = len(pairs) >= min_his_cation_pairs
        except Exception as e:
            print(f"  Error detecting motifs in {pdb_path}: {e}")
            continue
        
        # Step 2: Score with PyRosetta (or stub)
        try:
            if use_stub:
                scores = score_interface_destabilization_stub(pdb_path)
            else:
                scores = score_interface_destabilization(pdb_path)
            result['scores'] = scores
            result['passes_elec'] = scores.get('ddG_elec', -999) >= min_ddg_elec
        except Exception as e:
            print(f"  Error scoring {pdb_path}: {e}")
            continue
        
        # Step 3: Check if passes all criteria
        result['passes_all'] = result['passes_motif'] and result['passes_elec']
        results.append(result)
    
    # Summary
    passing = [r for r in results if r['passes_all']]
    print(f"\nFiltering complete:")
    print(f"  Total candidates: {len(results)}")
    print(f"  Passing motif criterion: {sum(r['passes_motif'] for r in results)}")
    print(f"  Passing elec criterion: {sum(r['passes_elec'] for r in results)}")
    print(f"  Passing all criteria: {len(passing)}")
    
    return results


# Example: Test filtering pipeline with available structures
print("Testing filtering pipeline...")
print("\nFiltering criteria:")
print("  1. >= 1 His-Arg/Lys pair at interface (within 3.5Å)")
print("  2. ddG_elec >= 0 (electrostatic destabilization at low pH)")

# Test with available PDBs
test_pdbs = [TARGET_PCSK9] if os.path.exists(TARGET_PCSK9) else []
if test_pdbs:
    print(f"\nTesting with: {test_pdbs}")
    results = filter_interface_destabilization_candidates_reference(
        test_pdbs, 
        min_his_cation_pairs=1, 
        min_ddg_elec=0.0,
        use_stub=True
    )
    
    for r in results:
        print(f"\n  {os.path.basename(r['pdb_path'])}:")
        print(f"    His-cation pairs: {r['num_pairs']}")
        print(f"    ddG_elec: {r['scores'].get('ddG_elec', 'N/A'):.2f}")
        print(f"    Passes: {'✓' if r['passes_all'] else '✗'}")
else:
    print("\nNo test PDBs available.")


---
# Part 3: Monomer Destabilization with HBNet (Strategy B)

## Objective
Use PyRosetta HBNet to design buried His-containing hydrogen bond networks in a protein scaffold.

## Background
HBNet (Hydrogen Bond Network) is a Rosetta protocol for designing networks of hydrogen-bonded residues. For Strategy B, we want to install buried networks containing:
- His-His pairs
- His-Arg pairs  
- His-Lys pairs

These networks are stable at neutral pH but become destabilized when His becomes protonated at low pH.

## Data
- Scaffold: De novo 3-helix bundle (PDB 5L33)


In [ ]:
# Part 3: Setup
SCAFFOLD_PDB = os.path.join(DATA_DIR, "scaffold.pdb")  # PDB 5L33
print(f"Scaffold: {SCAFFOLD_PDB}")
print(f"File exists: {os.path.exists(SCAFFOLD_PDB)}")


## Task 3.1: Identify Core Residues for HBNet

First, identify buried core residues suitable for installing His-containing networks.
A residue is considered "buried" if its relative SASA < 25%.


In [ ]:
def identify_core_residues(pdb_path, sasa_threshold=0.25):
    """
    Identify buried core residues suitable for HBNet installation.
    
    Args:
        pdb_path: Path to PDB file
        sasa_threshold: Relative SASA threshold (residues below this are "buried")
    
    Returns:
        List of dictionaries with core residue info:
        [
            {'chain': str, 'resnum': int, 'resname': str, 'rel_sasa': float},
            ...
        ]
    """
    core_residues = []
    
    # TODO: Implement using FreeSASA or BioPython SASA
    # Steps:
    # 1. Load structure
    # 2. Calculate per-residue SASA
    # 3. Calculate relative SASA (SASA / max_SASA for that residue type)
    # 4. Filter for residues with relative SASA < threshold
    
    return core_residues


# =============================================================================
# REFERENCE IMPLEMENTATION
# =============================================================================

# Standard max SASA values (Å²) for amino acids (Tien et al. 2013)
MAX_SASA = {
    'ALA': 129.0, 'ARG': 274.0, 'ASN': 195.0, 'ASP': 193.0, 'CYS': 167.0,
    'GLN': 225.0, 'GLU': 223.0, 'GLY': 104.0, 'HIS': 224.0, 'ILE': 197.0,
    'LEU': 201.0, 'LYS': 236.0, 'MET': 224.0, 'PHE': 240.0, 'PRO': 159.0,
    'SER': 155.0, 'THR': 172.0, 'TRP': 285.0, 'TYR': 263.0, 'VAL': 174.0
}

def identify_core_residues_reference(pdb_path, sasa_threshold=0.25):
    """
    Reference implementation using BioPython's SASA calculation.
    
    Identifies buried core residues with relative SASA < threshold.
    """
    from Bio.PDB import PDBParser
    from Bio.PDB.SASA import ShrakeRupley
    
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("protein", pdb_path)
    model = structure[0]
    
    # Calculate SASA using Shrake-Rupley algorithm
    sr = ShrakeRupley()
    sr.compute(model, level="R")  # Compute at residue level
    
    core_residues = []
    
    for chain in model.get_chains():
        for residue in chain.get_residues():
            # Skip hetero residues
            if residue.id[0] != ' ':
                continue
            
            resname = residue.resname
            if resname not in MAX_SASA:
                continue
            
            # Get residue SASA (sum of atom SASAs)
            res_sasa = residue.sasa if hasattr(residue, 'sasa') else 0
            
            # Calculate relative SASA
            max_sasa = MAX_SASA[resname]
            rel_sasa = res_sasa / max_sasa if max_sasa > 0 else 0
            
            if rel_sasa < sasa_threshold:
                core_residues.append({
                    'chain': chain.id,
                    'resnum': residue.id[1],
                    'resname': resname,
                    'sasa': round(res_sasa, 1),
                    'rel_sasa': round(rel_sasa, 3)
                })
    
    return core_residues


# Test with scaffold structure
print("Testing core residue identification...")

if os.path.exists(SCAFFOLD_PDB):
    try:
        core = identify_core_residues_reference(SCAFFOLD_PDB, sasa_threshold=0.25)
        print(f"\nScaffold structure ({SCAFFOLD_PDB}):")
        print(f"  Total core residues (SASA < 25%): {len(core)}")
        
        if core:
            # Group by residue type
            from collections import Counter
            restype_counts = Counter(r['resname'] for r in core)
            print(f"\n  Core residue types:")
            for restype, count in restype_counts.most_common():
                print(f"    {restype}: {count}")
            
            # Show example buried residues
            print(f"\n  Example buried residues:")
            for res in core[:5]:
                print(f"    {res['resname']} {res['chain']}{res['resnum']}: "
                      f"SASA={res['sasa']:.1f}Å², rel={res['rel_sasa']:.1%}")
    except Exception as e:
        print(f"Error: {e}")
else:
    print(f"File not found: {SCAFFOLD_PDB}")


## Task 3.2: Configure and Run HBNet

Configure PyRosetta HBNet to find His-containing hydrogen bond networks in the protein core.


In [ ]:
def setup_hbnet_for_his_networks(pose, core_positions, network_types=["HIS_HIS", "HIS_ARG", "HIS_LYS"]):
    """
    Configure HBNet to design His-containing hydrogen bond networks.
    
    This function sets up PyRosetta's HBNet mover to find and design
    hydrogen bond networks containing histidine residues in the protein core.
    
    Args:
        pose: PyRosetta Pose object
        core_positions: List of residue positions (Rosetta numbering) to target
        network_types: Types of networks to design:
            - "HIS_HIS": His-His networks
            - "HIS_ARG": His-Arg networks  
            - "HIS_LYS": His-Lys networks
    
    Returns:
        Configured HBNet mover ready to apply
    
    Note:
        Requires PyRosetta with HBNet protocol available.
        Core positions should be pre-filtered for buried residues (SASA < 25%).
    
    Example:
        >>> import pyrosetta
        >>> pyrosetta.init("-mute all")
        >>> pose = pyrosetta.pose_from_pdb("scaffold.pdb")
        >>> core_positions = [10, 15, 20, 25, 30]  # From identify_core_residues
        >>> hbnet = setup_hbnet_for_his_networks(pose, core_positions)
        >>> hbnet.apply(pose)  # Designs networks in pose
    
    Key HBNet settings for pH-sensitive design:
        - allowed_aas: "HRK" (His, Arg, Lys)
        - min_network_size: 2 (at least 2 residues per network)
        - require_his: True (each network must contain His)
        - only_buried: True (target core positions)
    """
    # TODO: Implement HBNet configuration using PyRosetta
    # Required components:
    # 1. ResidueIndexSelector for core positions
    # 2. HBNet mover configuration
    # 3. Amino acid restrictions (H, R, K only)
    # 4. Network size requirements
    
    raise NotImplementedError(
        "This function requires PyRosetta with HBNet protocol. "
        "See: https://www.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/Movers/HBNet"
    )


# =============================================================================
# STUB returning mock HBNet configuration for testing
# =============================================================================
class HBNetStub:
    """Stub HBNet class for pipeline testing."""
    def __init__(self, core_positions, network_types):
        self.core_positions = core_positions
        self.network_types = network_types
        self.networks = []
    
    def apply(self, pose=None):
        """Mock apply that generates example networks."""
        import random
        # Generate mock networks
        for i in range(random.randint(1, 5)):
            # Pick 2-3 random core positions
            net_size = random.randint(2, min(3, len(self.core_positions)))
            positions = random.sample(self.core_positions, net_size)
            self.networks.append({
                'id': i + 1,
                'residues': positions,
                'type': random.choice(self.network_types),
                'hbond_energy': random.uniform(-1.5, -0.2)
            })
        return self.networks


def setup_hbnet_for_his_networks_stub(pose, core_positions, network_types=["HIS_HIS", "HIS_ARG", "HIS_LYS"]):
    """Stub implementation returning mock HBNet for testing."""
    return HBNetStub(core_positions, network_types)


print("HBNet configuration function defined (stub available for testing).")
print("\nHBNet network types for pH-sensitive design:")
print("  - HIS_HIS: Two histidines H-bonding")
print("  - HIS_ARG: Histidine H-bonding with arginine")
print("  - HIS_LYS: Histidine H-bonding with lysine")


In [ ]:
def score_hbnet_designs(pose, networks, min_his_hbond_energy=-0.5):
    """
    Score HBNet designs and filter based on His H-bond energy.
    
    The key criterion is that histidine residues in the network must have
    sufficiently strong H-bonds (energy < -0.5 kcal/mol) to be functional
    at neutral pH but disrupted when His becomes protonated at low pH.
    
    Args:
        pose: PyRosetta Pose with HBNet-designed networks
        networks: List of network definitions from HBNet, each containing:
            - 'residues': list of residue numbers in network
            - 'type': network type (HIS_HIS, HIS_ARG, HIS_LYS)
        min_his_hbond_energy: Energy threshold in kcal/mol (default -0.5)
            More negative = stronger H-bond required
    
    Returns:
        List of passing networks with scores:
        [
            {
                'network_id': int,
                'residues': list,
                'type': str,
                'his_hbond_energy': float,
                'passes': bool
            },
            ...
        ]
    
    Note:
        Requires PyRosetta for actual H-bond energy extraction.
        Uses HBondSet to analyze hydrogen bonds in the pose.
    
    Example:
        >>> # After running HBNet
        >>> networks = hbnet.get_networks()
        >>> passing = score_hbnet_designs(pose, networks, min_his_hbond_energy=-0.5)
        >>> print(f"Found {len(passing)} networks with strong His H-bonds")
    """
    passing_networks = []
    
    # TODO: Implement scoring using PyRosetta HBondSet
    # Required steps:
    # 1. Extract HBondSet from scored pose
    # 2. For each network, identify His residues
    # 3. Find H-bonds involving His ND1/NE2 atoms
    # 4. Sum energies and compare to threshold
    
    raise NotImplementedError(
        "This function requires PyRosetta for H-bond energy extraction. "
        "See: HBondSet in pyrosetta.rosetta.core.scoring.hbonds"
    )


# =============================================================================
# STUB for scoring (works with HBNetStub)
# =============================================================================
def score_hbnet_designs_stub(pose, networks, min_his_hbond_energy=-0.5):
    """
    Stub implementation for testing the scoring pipeline.
    Works with HBNetStub networks.
    """
    results = []
    
    for i, net in enumerate(networks):
        # Networks from stub have 'hbond_energy' pre-computed
        if isinstance(net, dict):
            energy = net.get('hbond_energy', -0.3)
            residues = net.get('residues', [])
            net_type = net.get('type', 'UNKNOWN')
        else:
            energy = -0.3
            residues = []
            net_type = 'UNKNOWN'
        
        passes = energy < min_his_hbond_energy
        
        results.append({
            'network_id': i + 1,
            'residues': residues,
            'type': net_type,
            'his_hbond_energy': round(energy, 3),
            'passes': passes
        })
    
    return results


print("HBNet scoring function defined (stub available for testing).")
print(f"\nFiltering criterion: His H-bond energy < -0.5 kcal/mol")
print("  - More negative energy = stronger H-bond")
print("  - Strong H-bonds at neutral pH → disrupted when His protonates")


## Task 3.4: Complete Monomer Destabilization Pipeline

Combine all steps into a complete pipeline.


In [ ]:
def run_monomer_destabilization_pipeline(pdb_path, output_dir="hbnet_output"):
    """
    Complete pipeline for monomer destabilization design using HBNet.
    
    Workflow:
    1. Load scaffold structure
    2. Identify core residues (SASA < 25%)
    3. Run HBNet to design His-containing networks
    4. Score and filter networks (His H-bond energy < -0.5 kcal/mol)
    5. Output passing designs
    
    Args:
        pdb_path: Path to scaffold PDB
        output_dir: Directory for output files
    
    Returns:
        Dictionary with pipeline results:
        {
            'scaffold': str,
            'core_residues': list,
            'networks_found': int,
            'networks_passing': int,
            'passing_designs': list
        }
    
    Note:
        Requires PyRosetta with HBNet protocol.
        For testing, use run_monomer_destabilization_pipeline_stub().
    """
    # TODO: Implement complete pipeline with PyRosetta
    raise NotImplementedError("Requires PyRosetta. Use stub for testing.")


# =============================================================================
# REFERENCE IMPLEMENTATION using stubs for testing
# =============================================================================
def run_monomer_destabilization_pipeline_reference(pdb_path, output_dir="hbnet_output", 
                                                    sasa_threshold=0.25,
                                                    min_his_hbond_energy=-0.5):
    """
    Reference implementation using stub functions for pipeline testing.
    """
    print(f"=" * 60)
    print(f"Running Monomer Destabilization Pipeline")
    print(f"=" * 60)
    print(f"Input scaffold: {pdb_path}")
    print(f"Output directory: {output_dir}")
    print()
    
    results = {
        'scaffold': pdb_path,
        'core_residues': [],
        'networks_found': 0,
        'networks_passing': 0,
        'passing_designs': []
    }
    
    # Step 1: Identify core residues
    print("Step 1: Identifying core residues (SASA < 25%)...")
    try:
        core = identify_core_residues_reference(pdb_path, sasa_threshold)
        core_positions = [r['resnum'] for r in core]
        results['core_residues'] = core
        print(f"  Found {len(core)} core residues")
    except Exception as e:
        print(f"  Error: {e}")
        return results
    
    if len(core_positions) < 3:
        print("  WARNING: Too few core residues for HBNet")
        return results
    
    # Step 2: Setup and run HBNet (using stub)
    print("\nStep 2: Running HBNet for His-containing networks...")
    hbnet = setup_hbnet_for_his_networks_stub(
        pose=None,  # Stub doesn't need real pose
        core_positions=core_positions,
        network_types=["HIS_HIS", "HIS_ARG", "HIS_LYS"]
    )
    networks = hbnet.apply()
    results['networks_found'] = len(networks)
    print(f"  Found {len(networks)} potential networks")
    
    # Step 3: Score networks
    print(f"\nStep 3: Scoring networks (threshold: His H-bond < {min_his_hbond_energy} kcal/mol)...")
    scored = score_hbnet_designs_stub(None, networks, min_his_hbond_energy)
    passing = [n for n in scored if n['passes']]
    results['networks_passing'] = len(passing)
    results['passing_designs'] = passing
    
    print(f"  Passing networks: {len(passing)} / {len(networks)}")
    
    # Step 4: Summary
    print(f"\n{'=' * 60}")
    print("Pipeline Summary")
    print(f"{'=' * 60}")
    print(f"  Core residues: {len(core)}")
    print(f"  Networks found: {len(networks)}")
    print(f"  Networks passing: {len(passing)}")
    
    if passing:
        print(f"\n  Passing network details:")
        for net in passing:
            print(f"    Network {net['network_id']}: {net['type']}")
            print(f"      Residues: {net['residues']}")
            print(f"      His H-bond energy: {net['his_hbond_energy']:.3f} kcal/mol")
    
    return results


# Test the complete pipeline with scaffold
print("Testing complete monomer destabilization pipeline...")

if os.path.exists(SCAFFOLD_PDB):
    results = run_monomer_destabilization_pipeline_reference(
        SCAFFOLD_PDB,
        sasa_threshold=0.25,
        min_his_hbond_energy=-0.5
    )
else:
    print(f"Scaffold not found: {SCAFFOLD_PDB}")


---
# Summary

This task covered the key computational workflows for pH-sensitive binder design:

## Part 0: Conceptual Understanding
- Histidine's role (pKa ~6.0)
- His-cation motifs for interface destabilization
- Buried His networks for monomer destabilization

## Part 1: ProteinMPNN with Histidine Bias
- Global bias with `--bias_AA "H:2.0"`
- Position-specific bias with JSON dictionaries

## Part 2: Interface Destabilization (Strategy A)
- His-Arg/Lys motif detection at interface
- PyRosetta FastRelax and InterfaceAnalyzer
- Filter: His-cation pair + ddG_elec >= 0

## Part 3: Monomer Destabilization (Strategy B)
- Core residue identification (SASA < 25%)
- HBNet for His-His, His-Arg, His-Lys networks
- Filter: His H-bond energy < -0.5 kcal/mol
